In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [2]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
datadir = './data'
is_tensorboard = True

In [3]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (18225)kill: (18231): Aucun processus de ce type

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


#### Increase of train dataset against test dataset.

In [5]:
import numpy as np
x_train = np.concatenate([x_train, x_test[:27,:,:,:]], axis=0)
x_test = x_test[27:,:,:,:]
print(x_train.shape, x_test.shape)

y_train = np.concatenate([y_train, y_test[:27]])
y_test = y_test[27:]
print(y_train.shape, y_test.shape)

(441, 224, 224, 3) (20, 224, 224, 3)
(441,) (20,)


In [52]:
import p8_util_config
import NNAdaNetBuilder

#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------
if p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'CNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['conv_layer_num']
elif p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'RNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['rnn_layer_num']

print("\n Number of convolutional layers= {}".format(layer_num))
oNNAdaNetBuilder = NNAdaNetBuilder.NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=layer_num)


 Number of convolutional layers= 1

*** NNAdaNetBuilder : NN Type=GRU


In [53]:
oNNAdaNetBuilder.show()



NN type              : ............................ GRU
Units in dense layer : ............................ 10
Number of layers     : ............................ 1
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Weights initializer  : ............................ truncated_normal
Batch normalization  : ............................ True


Hidden units         : ............................ 128
Stacked cells        : ............................ 1
Time steps           : ............................ 224


In [54]:
p8_util_config.MAX_STEPS,p8_util_config.NUM_EPOCHS

(200, 6)

In [55]:
# Parameters that will be provided to model_fn
import p8_util_config
nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']


params = {'net_builder':oNNAdaNetBuilder, 'nn_type':nn_type}
print("\n NN Type= {}".format(nn_type))
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)  

classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)


 NN Type= GRU

*** make_config() : output dir= ./tmp/baseline/GRU
INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/GRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f342de979e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [56]:
train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, p8_util_config.NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.MAX_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, p8_util_config.NUM_EPOCHS\
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        throttle_secs=1)

In [58]:
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 1)

*** my_model_fn() : predicted_classes shape= (?,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./tmp/baseline/GRU/model.ckpt.

*** generator() : labels shape= (441,) / label values= 0
INFO:tensorflow:loss = 1.0994158, step = 1

*** generator() : labels shape= (441,) / label values= 0
INFO:tensorflow:Saving checkpoints for 10 into ./tmp/basel

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 80: ./tmp/baseline/GRU/model.ckpt-80

*** generator() : labels shape= (441,) / label values= 0
INFO:tensorflow:Saving checkpoints for 90 into ./tmp/baseline/GRU/model.ckpt.
feature_shape= [224, 224, 3]
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 1)

*** my_model_fn() : predicted_classes shape= (?,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-06-22:37:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/GRU/model.ckpt-90
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (20,) / label values= 2
INFO:tensorflow:Finished evaluation at 2019-05-06-22:37:30
INFO:tensorflow:Saving dict for global step 90: GRU_Eval_accuracy = 0.4, global_step = 90, loss = 1.212925
INFO:tensorflow:Saving 'checkpoint_path' summary for glob

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (20,) / label values= 2
INFO:tensorflow:Finished evaluation at 2019-05-06-22:42:22
INFO:tensorflow:Saving dict for global step 170: GRU_Eval_accuracy = 0.5, global_step = 170, loss = 1.5005395
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 170: ./tmp/baseline/GRU/model.ckpt-170

*** generator() : labels shape= (441,) / label values= 0
INFO:tensorflow:Saving checkpoints for 180 into ./tmp/baseline/GRU/model.ckpt.
feature_shape= [224, 224, 3]
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 1)

*** my_model_fn() : predicted_classes shape= (?,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-06-22:42:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/GRU/model.ckpt-180
INFO:tensorflow:Running local_init_op.
INFO:tensor

In [59]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 1056.6603984832764
GRU_EVAL_ACCURACY: 0.5
LOSS: 1.4151865243911743
GLOBAL_STEP: 200


In [16]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')